In [8]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [9]:
from FoliumPlot import folium_plot, folium_layout
from Functions import *

## BASE MAPS

### administrative map

In [10]:
# map path and layer path
map_adm_path = "map/gadm36_FIN_shp.zip"
map_adm_region_layer = "gadm36_FIN_2.shp"
map_adm_city_layer = "gadm36_FIN_3.shp"
map_adm_district_layer = "gadm36_FIN_4.shp"

In [11]:
# import and filter
filter_col_map_adm = ["NAME_1", "NAME_2", "NAME_3", "NAME_4", "geometry"]
map_adm_region = gpd.read_file(
    "zip://./" + map_adm_path + "!/" + map_adm_region_layer
).filter(filter_col_map_adm)

map_adm_city = gpd.read_file(
    "zip://./" + map_adm_path + "!/" + map_adm_city_layer
).filter(filter_col_map_adm)

map_adm_district = gpd.read_file(
    "zip://./" + map_adm_path + "!/" + map_adm_district_layer
).filter(filter_col_map_adm)

In [12]:
# data validation
print(map_adm_region.shape, map_adm_city.shape, map_adm_district.shape)

(21, 3) (80, 4) (437, 5)


In [13]:
# Helsinki center adm map
filter_map_adm_helsinki = ["Helsinki", "Vantaa", "Espoo"]
map_adm_helsinki = map_adm_district.query(
    "NAME_4 in @filter_map_adm_helsinki"
).reset_index(drop=True)

### postcode map

In [14]:
# map path and layer path
map_pc_path = 'map/paavo_postinumeroalueet_2020.zip'
map_pc_layer = 'paavo_postinumeroalueet_2020.zip.shp'

In [15]:
# import and filter
filter_col_map_pc = ["posti_alue", "geometry"]
map_pc_raw = gpd.read_file("zip://./" + map_pc_path + "!/" + map_pc_layer).filter(
    filter_col_map_pc
)

In [16]:
# add adm data through sjoin
map_pc_adm = gpd.sjoin(
    gpd.GeoDataFrame(
        map_pc_raw.drop("geometry", axis=1), geometry=map_pc_raw.geometry.centroid
    ),
    map_adm_district.to_crs(map_pc_raw.crs),
).drop(["index_right", "geometry"], axis=1)

In [17]:
# merge data to gdf
col_map_pc_merge = "posti_alue"
map_pc_geo = map_pc_raw.merge(map_pc_adm, on=col_map_pc_merge).to_crs(4326)

In [19]:
map_pc_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2960 entries, 0 to 2959
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   posti_alue  2960 non-null   object  
 1   geometry    2960 non-null   geometry
 2   NAME_1      2960 non-null   object  
 3   NAME_2      2960 non-null   object  
 4   NAME_3      2960 non-null   object  
 5   NAME_4      2960 non-null   object  
dtypes: geometry(1), object(5)
memory usage: 161.9+ KB


In [19]:
# save
output_path = "map/map_pc_geo"
map_pc_geo.to_file(output_path)

In [20]:
# Helsinki center pc map
map_pc_helsinki = map_pc_geo.query("NAME_4 in @filter_map_adm_helsinki").reset_index(
    drop=True
)

In [21]:
# save
output_path = 'map/map_pc_helsinki'
map_pc_helsinki.to_file(output_path)

In [25]:
# folium plot
m = folium_plot(
    gdf=map_pc_helsinki, col="NAME_4", tooltip_col="posti_alue", fill_opacity=0.8
)

### engineering parameter

In [26]:
# ep site
data_ep_path = 'data/ep/EP_SITE.csv'
data_ep_raw = pd.read_csv(data_ep_path)

In [27]:
# convert df to gdf
data_ep_site_geo = gpd.GeoDataFrame(
    data_ep_raw,
    geometry=gpd.points_from_xy(
        x=data_ep_raw.Longitude, y=data_ep_raw.Latitude, crs=4326
    ),
    copy=True,
)

In [28]:
# ep city
data_ep_city_raw = (
    gpd.sjoin(data_ep_site_geo, map_adm_city)
    .drop(["index_right", "Longitude", "Latitude"], axis=1)
    .groupby(["NAME_1", "NAME_2", "NAME_3"], as_index=False)
    .sum()
)
# 5G percentage
data_ep_city_raw = data_ep_city_raw.assign(
    Site_5G_pct=lambda x: np.round(x.Swapped_5G / x.Total_Sites, 2)
)

In [29]:
# merge data to postcode map
data_ep_city_geo = map_adm_city.merge(
    data_ep_city_raw, on=["NAME_1", "NAME_2", "NAME_3"]
)

In [31]:
data_ep_city_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 62 entries, 0 to 61
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   NAME_1         62 non-null     object  
 1   NAME_2         62 non-null     object  
 2   NAME_3         62 non-null     object  
 3   geometry       62 non-null     geometry
 4   Total_Sites    62 non-null     int64   
 5   Swapped_Sites  62 non-null     int64   
 6   Swapped_5G     62 non-null     int64   
 7   Site_5G_pct    62 non-null     float64 
dtypes: float64(1), geometry(1), int64(3), object(3)
memory usage: 4.4+ KB


In [30]:
# save gdf data
output_path = 'data/ep/data_ep_city_geo'
data_ep_city_geo.to_file(output_path)

<ipython-input-30-17dd5744768c>:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  data_ep_city_geo.to_file(output_path)


In [36]:
# save gdf column name
output_col_name_path = "data/ep/data_ep_city_geo/col_name.csv"
pd.DataFrame(data_ep_city_geo.drop("geometry", axis=1).columns).to_csv(
    output_col_name_path, index=False
)

In [32]:
# folium plot
m = folium_plot(
    gdf=data_ep_city_geo[data_ep_city_geo["Site_5G_pct"] > 0],
    col="Site_5G_pct",
    n_col=7,
    fill_opacity=1.0,
    palette_n={"Oranges": [0.01, 0.05, 0.10, 0.25, 0.50]},
    color="lightgrey",
    bin_decimal=2,
)

### population map

In [34]:
# data path
map_population_path = "map/population_1km/vaki2019_1km_kp.shp"
map_population_raw = gpd.read_file(map_population_path).to_crs(4326)

In [35]:
map_population_raw.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 98703 entries, 0 to 98702
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   kunta      98703 non-null  object  
 1   grd_id     98703 non-null  object  
 2   id_nro     98703 non-null  float64 
 3   vaesto     98703 non-null  float64 
 4   miehet     98703 non-null  float64 
 5   naiset     98703 non-null  float64 
 6   ika_0_14   98703 non-null  float64 
 7   ika_15_64  98703 non-null  float64 
 8   ika_65_    98703 non-null  float64 
 9   xkoord     98703 non-null  int64   
 10  ykoord     98703 non-null  int64   
 11  geometry   98703 non-null  geometry
dtypes: float64(7), geometry(1), int64(2), object(2)
memory usage: 9.0+ MB
